In [ ]:
import random

import sets
import threading

import Tkinter as tkinter
import tkFileDialog as tkfiledialog

#run numruns times and make new file for each
import sys
#numruns = 1000
#set the number of words to add to the existing network
#num_desired_words = 50
ENTRY_WIDTH = 20


def filename(j):
    j = j+1
    str_list = ['run']
    str_list.append('j')
    str_list.append('.txt')
    return ''.join(str_list)

def is_present(target_connection, dataset): # returns a true/false list
  # indicating whether the kid has the edge of the full graph given their
  # vocabulary
    are_equal_tuples = lambda x, y: x[0] == y[0] and x[1] == y[1]
    for connection in dataset:
        if are_equal_tuples(connection, target_connection):
            return True
    return False

def check_if_in_dictionary(candidate_word, datalist, all_possible_wordslist):
    #make sure the word selected exists in the 30_mo_dictionary
    while not candidate_word in all_possible_wordslist:
        random_pair = random.choice(datalist)
        candidate_word = random_pair[0]
        if(candidate_word in all_possible_wordslist):
            break
    new_candidate_word = candidate_word
    return new_candidate_word

def run_growth(numruns, num_desired_words, given_vocab_loc, all_words_loc,
    sum_file_loc, on_iterate, on_error, on_done):

    try:

        k=0
        run = ''
        summ_file = open(sum_file_loc, "w")
        summ_dictionary = {}

        print k
        print numruns
        print k < numruns

        while k < numruns:

            on_iterate(k, numruns)

            #run = filename(k)
            #file = open(run, "w")

            allwords = open(all_words_loc, 'r')
            combinations = [item.rstrip('\n') for item in allwords]
            combinations = map(lambda x: x.split(","), combinations)
            allwords.close()
            connection_map = {}

            all_possible_wordslist = []

            # Map out the connections between words
            for x in combinations:
               if not x[0] in connection_map:
                   connection_map[x[0]] = []
                   all_possible_wordslist.append(x[0])
               if not x[1] in connection_map:
                   connection_map[x[1]] = []
               connection_map[x[0]].append(x[1])
               connection_map[x[1]].append(x[0])
            # Remove duplicates
            for key in connection_map.keys():
                edgesList = connection_map[key]
                edgesSet = sets.Set(edgesList)
                connection_map[key] = edgesSet

            # Load up existing connections for given child
            knownwords = open(given_vocab_loc, 'r')
            datalist = knownwords.readlines()
            knownwords.close()
            datalist = [item.rstrip('\n').split(",") for item in datalist]

            # Randomly fill network with unique word
            num_added_word = 0
            while num_added_word < num_desired_words:
                #choose a random tuple
                random_pair = random.choice(datalist)
                candidate_word = random_pair[0]

                candidate_word = check_if_in_dictionary(candidate_word, datalist, all_possible_wordslist)

                while (len(connection_map[candidate_word]) == 1):
                    #print 'only cnx to self', candidate_word
                    random_pair = random.choice(datalist)
                    candidate_word = random_pair[0]
                    candidate_word = check_if_in_dictionary(candidate_word, datalist, all_possible_wordslist)

                #reassign 
                lucky_word = candidate_word
                
                #randomly select a word from possible words to be added as a neighbor
                connection_list = []
                for x in connection_map[lucky_word]:
                    connection_list.append(x)
                candidate_new_word = random.choice(connection_list)

                if not is_present((lucky_word , candidate_new_word), datalist):
                    #acquire new_word here for word being added
                    #if item in connection_map[new_word] exists 
                    #in datalist add edge
                    new_word = candidate_new_word
                    new_word_edgelist = connection_map[new_word]

                    for connection in datalist:
                        if connection[0] in new_word_edgelist and not is_present((new_word, connection[0]), datalist):
                            datalist.append((new_word, connection[0]))
                            datalist.append((connection[0], new_word))

                    if not new_word in summ_dictionary:
                        summ_dictionary[new_word] = 1
                    else:
                        summ_dictionary[new_word] += 1
                        
                    #print "Added: ", new_word, "by", lucky_word,  "giving tuple", (new_word, lucky_word)
                    num_added_word+=1

            print "on run ", k
            #file.write(str(datalist))
            #file.close()
            k = k+1

        summ_items = summ_dictionary.items() # List of tuples = (key, value)
        summ_items.sort(key=lambda x: x[1], reverse = True)

        for word, count in summ_items:
            summ_file.write(word)
            summ_file.write(',')
            summ_file.write(str(count))
            summ_file.write('\n')
        summ_file.close()
        
        on_done()

    except Exception as e:
        on_error(e)


def browse_for_entry(entry_box, dialog_type, filetype):
    """Browse for a file and put that filename in an entry box.
 
    @param entry_box: The entry box to put the filename in.
    @type entry_box: tkinter.Entry
    @param dialog_type: The type of dialog to display. Should be "open" or
        "save as"
    @type dialog_type: str
    """
    dialog_type = dialog_type.lower()
    if dialog_type == "open":
        name = tkfiledialog.askopenfilename(
            filetypes=[filetype]
        )
    elif dialog_type == "saveas":
        name = tkfiledialog.asksaveasfilename(
            filetypes=[filetype]
        )
    else:
        raise ValueError("%s not a valid dialog type." % dialog_type)
    entry_box.delete(0, tkinter.END)
    entry_box.insert(0, str(name))


def show_finished(status_label, run_button):
    run_button.configure(text="Run Again", state=tkinter.NORMAL)
    status_label.configure(text="Done.")

def run_growth_thread(number_runs_entry, number_desired_words_entry,
    child_vocabulary_entry, all_words_entry, output_summary_entry, status_label,
    run_button):
    
    number_runs = int(number_runs_entry.get())
    number_desired_words = int(number_desired_words_entry.get())
    child_vocabulary = child_vocabulary_entry.get()
    all_words = all_words_entry.get()
    output_summary = output_summary_entry.get()

    on_iterate = lambda current, total: status_label.configure(
        text="%d / %d Iterations" % (current, total)
    )

    on_error = lambda err: status_label.configure(
        text="ERROR: %s" % str(err)
    )

    on_done = lambda: show_finished(status_label, run_button)

    thread_func = lambda: run_growth(
        number_runs,
        number_desired_words,
        child_vocabulary,
        all_words,
        output_summary,
        on_iterate,
        on_error,
        on_done
    )

    run_button.configure(text="Running. Please wait.", state=tkinter.DISABLED)

    job_thread = threading.Thread(target=thread_func)
    job_thread.start()


if __name__ == '__main__':

    master = tkinter.Tk()
    master.title("Daxlab Easy Growth Runner")
    tkinter.Label(master, text="Number runs:").grid(row=0, column=0, sticky=tkinter.E)
    tkinter.Label(master, text="Number words added:").grid(row=1, column=0, sticky=tkinter.E)
    tkinter.Label(master, text="Child\'s vocabulary:").grid(row=2, column=0, sticky=tkinter.E)
    tkinter.Label(master, text="All words:").grid(row=3, column=0, sticky=tkinter.E)
    tkinter.Label(master, text="Output summary:").grid(row=4, column=0, sticky=tkinter.E)

    number_runs_entry = tkinter.Entry(master)
    number_runs_entry.grid(row=0, column=1, sticky=tkinter.E+tkinter.W)
    number_runs_entry.insert(0, "1000")

    number_desired_words_entry = tkinter.Entry(master)
    number_desired_words_entry.grid(row=1, column=1, sticky=tkinter.E+tkinter.W)
    number_desired_words_entry.insert(0, "50")

    child_vocabulary_entry = tkinter.Entry(master)
    child_vocabulary_entry.grid(row=2, column=1, sticky=tkinter.E+tkinter.W)

    all_words_entry = tkinter.Entry(master)
    all_words_entry.grid(row=3, column=1, sticky=tkinter.E+tkinter.W)

    output_summary_entry = tkinter.Entry(master)
    output_summary_entry.grid(row=4, column=1, sticky=tkinter.E+tkinter.W)

    text_file_desc = ("Text files", ".txt")
    csv_file_desc = ("CSV files", ".csv")

    tkinter.Button(
        master,
        text="Browse",
        command=lambda: browse_for_entry(child_vocabulary_entry, "open", text_file_desc)
    ).grid(row=2, column=2)

    tkinter.Button(
        master,
        text="Browse",
        command=lambda: browse_for_entry(all_words_entry, "open", text_file_desc)
    ).grid(row=3, column=2)

    tkinter.Button(
        master,
        text="Browse",
        command=lambda: browse_for_entry(output_summary_entry, "saveas", csv_file_desc)
    ).grid(row=4, column=2)


    status_label = tkinter.Label(master, text="ready...", width=ENTRY_WIDTH)
    status_label.grid(row=6, column=0, columnspan=3, sticky=tkinter.E+tkinter.W)

    run_button = tkinter.Button(master, text="Grow Network")
    run_button.grid(
        row=5,
        column=0,
        columnspan=3,
        sticky=tkinter.E+tkinter.W
    )

    run_cmd = lambda: run_growth_thread(
        number_runs_entry,
        number_desired_words_entry,
        child_vocabulary_entry,
        all_words_entry,
        output_summary_entry,
        status_label,
        run_button
    )
    run_button.configure(command=run_cmd)

    master.mainloop()

####  need to take our chosen kid, get this code running to generate summary
# runs. take the summary runs and compute a likelihood given the model